# item to category

## import modules and predifine utility functions

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import json
import re
import sys
from itertools import chain
from tqdm import tqdm
import re

In [2]:
def remove_paran(remove_text_list):
    return [re.sub(r'\([^)]*\)', '', a) for a in remove_text_list]

In [3]:
def paran_to_comma(remove_text_list):
    remove_text_list =  [a.rstrip() for a in remove_text_list]
    remove_text_list =  [re.sub(r'\(', ', ', a) for a in remove_text_list]
    remove_text_list =  [re.sub(r'\),', ',', a) for a in remove_text_list]
    return  [re.sub(r'\)', ', ', a) for a in remove_text_list]

## Category.xlsx preprocessing

In [4]:
category_df = pd.read_excel('/home/jun0/Projects/emart24/data/category.xlsx',engine='openpyxl') # pip install openpyxl
category_df.columns = ['KAN_CODE', '전체분류', '현재분류명', '상품예시']
category_df.head()

,KAN_CODE,전체분류,현재분류명,상품예시
0,1010101,가공식품>조미료>종합조미료>천연/발효조미료,천연/발효조미료,"다시다, 미원, 멸치가루, 버섯가루"
1,1010102,가공식품>조미료>종합조미료>식초,식초,"과일초, 발효초, 감식초, 사과식초, 현미식초, 홍초"
2,1010103,가공식품>조미료>종합조미료>천일염,천일염,"구운천일염, 꽃소금, 굵은소금, 가는소금, 볶은소금"
3,1010104,가공식품>조미료>종합조미료>가공염,가공염,"정제염, 맛소금, 허브솔트, 깨소금"
4,1010105,가공식품>조미료>종합조미료>설탕,설탕,"슈가파우더, 과당, 각설탕, 황설탕, 백설탕, 흑설탕"


In [5]:
# 한글자 상품예시 삭제
category_df['상품예시'] = [ [ b for b in a if len(b)>1 ] for a in [  a.split(', ') for a in tqdm(paran_to_comma(category_df['상품예시'])) ] ] 
#category_df['상품예시'] = [a.split(', ') for a in tqdm(paran_to_comma(category_df['상품예시']))]
category_df.head(100)
# print(category_df['상품예시'][764])

100%|██████████| 3062/3062 [00:00<00:00, 838367.96it/s]


,KAN_CODE,전체분류,현재분류명,상품예시
0,1010101,가공식품>조미료>종합조미료>천연/발효조미료,천연/발효조미료,"[다시다, 미원, 멸치가루, 버섯가루]"
1,1010102,가공식품>조미료>종합조미료>식초,식초,"[과일초, 발효초, 감식초, 사과식초, 현미식초, 홍초]"
2,1010103,가공식품>조미료>종합조미료>천일염,천일염,"[구운천일염, 꽃소금, 굵은소금, 가는소금, 볶은소금]"
3,1010104,가공식품>조미료>종합조미료>가공염,가공염,"[정제염, 맛소금, 허브솔트, 깨소금]"
4,1010105,가공식품>조미료>종합조미료>설탕,설탕,"[슈가파우더, 과당, 각설탕, 황설탕, 백설탕, 흑설탕]"
...,...,...,...,...
95,1040203,가공식품>수산가공식품>해조류>한천,한천,"[한천가루, 가루한천, 분말한천]"
96,1040204,가공식품>수산가공식품>해조류>다시마,다시마,"[건다시마, 마른다시마, 자른다시마]"
97,1040205,가공식품>수산가공식품>해조류>매생이,매생이,"[생매생이, 건매생이, 냉동매생이, 동결건조매생이]"
98,1040206,가공식품>수산가공식품>해조류>해초샐러드,해초샐러드,"[샐러드용해초, 해초세트, 샐러드해초세트]"


In [6]:
# 전체 분류명과 현재 분류명 일치하는지 확인
total = [a.split('>') for a in category_df['전체분류']]
current = category_df['현재분류명']
counter=0
for i in range(len(total)):
    if total[i][-1] != current[i]:
        counter+=1
if counter==0:
    print("전체 분류명과 현재 분류명 일치. ")
else:
    print("전체 분류명과 현재 분류명 불일치, 작업 필요")

전체 분류명과 현재 분류명 일치. 


## item data preprocessing

In [7]:
ROOT = Path('/home/jun0/Projects/emart24')
item = pd.read_csv(ROOT / 'data/prep/02.설명이추가된 상품마스터_20210325.csv', sep='|', dtype=str)
item['점포발주입수수량'] = item['점포발주입수수량'].fillna(0)
item['공급가'] = item['공급가'].fillna(0).apply(lambda x: round(float(x)))
item['매가'] = item['매가'].fillna(0)
item['점포유통기한일수'] = item['점포유통기한일수'].fillna(0)
item = item.astype({'점포발주입수수량': int, '공급가': int, '매가': int, '점포유통기한일수': int})
# create two more columns
item['현재분류명'] = ""
item["전체분류"] = ""
item["상품예시"] = ""
print(item.shape)
item.head()

(49299, 49)


,PLU_CD,상품명,대분류명,중분류명,소분류명,카테고리코드,상품속성코드,상품속성명,판매가능여부,시작일자,...,WED_YN,THU_YN,FRI_YN,SAT_YN,SUN_YN,GDS_DSCRT,GDS_SLPNT_DSCRT,현재분류명,전체분류,상품예시
0,9414453003319,KGB 크랜베리맛,자점,자점,자점,1151551055,401,일반,예,20140101,...,0,0,0,0,0,NaN,NaN,,,
1,9331275008295,레돈도)카푸치노125g,과자/빵,과자행사,비스켓행사(빼빼로),2092112004,401,일반,예,20200918,...,1,1,1,1,0,1. 20년 빼빼로데이 기획상품 2. 틴 케이스에 웨이퍼롤이 담겨 있는 상품,1. 틴 케이스에 웨이퍼롤이 담겨 있는 상품,,,
2,9331275008271,수입)레돈도(바닐라)150g,자점,자점,자점,1151551055,401,일반,예,20140101,...,0,0,0,0,0,NaN,NaN,,,
3,9331275002996,레돈도딸기&크림,자점,자점,자점,1151551055,401,일반,예,20140101,...,0,0,0,0,0,NaN,NaN,,,
4,9312931607557,쉬크)퀴트로4중 면도날(4입),자점,자점,자점,1151551055,401,일반,예,20140101,...,0,0,0,0,0,NaN,NaN,,,


In [8]:
print("column names of item df",item.columns)


column names of item df Index(['PLU_CD', '상품명', '대분류명', '중분류명', '소분류명', '카테고리코드', '상품속성코드', '상품속성명',
       '판매가능여부', '시작일자', '종료일자', '사용여부', '상온저온유형코드', '상온저온유형명', '시즌상품유형코드',
       '시즌상품유형명', '행사상품유형코드', '행사상품유형명', '특별상품유형코드', '특별상품유형명', '낱개상품코드',
       '신상품주기코드', '신상품주기명', '신상품시작일자', '신상품종료일자', '점포발주여부', '점포발주가능일자',
       '발주권장유형코드', '발주권장유형명', '반품유형코드', '반품유형명', '점포발주입수수량', '공급가', '매가',
       '점포유통기한일수', 'TRANL_CD', 'LT', 'MON_YN', 'TUE_YN', 'WED_YN', 'THU_YN',
       'FRI_YN', 'SAT_YN', 'SUN_YN', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT', '현재분류명',
       '전체분류', '상품예시'],
      dtype='object')


In [9]:
category_list=list(category_df['상품예시'])
new_item_df=pd.DataFrame(columns=item.columns)
new_item_df.head()

,PLU_CD,상품명,대분류명,중분류명,소분류명,카테고리코드,상품속성코드,상품속성명,판매가능여부,시작일자,...,WED_YN,THU_YN,FRI_YN,SAT_YN,SUN_YN,GDS_DSCRT,GDS_SLPNT_DSCRT,현재분류명,전체분류,상품예시


In [10]:

# pd.DataFrame(item.iloc[item_index], columns=item.columns)
for item_index in tqdm(range(len(item))):
    found_category_index=[]

    for category_iterator in range(len(category_list)):
        for i in range(len( category_list[category_iterator] )): 
            if category_list[category_iterator][i] in item["상품명"][item_index] : 
                found_category_index.append([category_iterator, i])
        temp_matrix = [1 if b in item["상품명"][item_index] else 0 for b in category_list[category_iterator]]
    '''
        if sum( [1 if b in item["상품명"][item_index] else 0 for b in category_list[category_iterator]] ) > 0 :
            found_category_index.append(category_iterator)
    found_category_index=list( set( found_category_index)) 
    if len(found_category_index) > 0:
        for found_index in range(len(found_category_index)):
            new_item_df = new_item_df.append(item.iloc[item_index], ignore_index=True)
            new_item_df.at[len(new_item_df)-1, "현재분류명"] = category_df["현재분류명"][found_category_index[found_index]]
            new_item_df.at[len(new_item_df)-1, "전체분류"] = category_df["전체분류"][found_category_index[found_index]]
    else: 
        new_item_df = new_item_df.append(item.iloc[item_index], ignore_index=True)
    '''

    if len(found_category_index) > 0:
        for found_index in range(len(found_category_index)):
            new_item_df = new_item_df.append(item.iloc[item_index], ignore_index=True)
            new_item_df.at[len(new_item_df)-1, "현재분류명"] = category_df["현재분류명"][found_category_index[found_index][0]]
            new_item_df.at[len(new_item_df)-1, "전체분류"] = category_df["전체분류"][found_category_index[found_index][0]]
            new_item_df.at[len(new_item_df)-1, "상품예시"] = category_df["상품예시"][found_category_index[found_index][0]][found_category_index[found_index][1]]
#    else: 
#        new_item_df = new_item_df.append(item.iloc[item_index], ignore_index=True)
    #if item_index >4:
    #    break
new_item_df.head(20)

100%|██████████| 49299/49299 [2:31:19<00:00,  5.43it/s]


,PLU_CD,상품명,대분류명,중분류명,소분류명,카테고리코드,상품속성코드,상품속성명,판매가능여부,시작일자,...,WED_YN,THU_YN,FRI_YN,SAT_YN,SUN_YN,GDS_DSCRT,GDS_SLPNT_DSCRT,현재분류명,전체분류,상품예시
0,9331275002996,레돈도딸기&크림,자점,자점,자점,1151551055,401,일반,예,20140101,...,0,0,0,0,0,NaN,NaN,딸기,신선식품>농산물>과일류>딸기,딸기
1,9312931607557,쉬크)퀴트로4중 면도날(4입),자점,자점,자점,1151551055,401,일반,예,20140101,...,0,0,0,0,0,NaN,NaN,면도날,일상용품>위생용품>면도위생용품>면도날,면도날
2,9312931607502,쉬크)퀴트로4중 면도기,자점,자점,자점,1151551055,401,일반,예,20140101,...,0,0,0,0,0,NaN,NaN,면도기,일상용품>위생용품>면도위생용품>면도기,면도기
3,9312931606550,쉬크)익스트림3 면도날,자점,자점,자점,1151551055,401,일반,예,20140101,...,0,0,0,0,0,NaN,NaN,면도날,일상용품>위생용품>면도위생용품>면도날,면도날
4,9312931606505,쉬크)익스트림3 면도기,자점,자점,자점,1151551055,401,일반,예,20140101,...,0,0,0,0,0,NaN,NaN,면도기,일상용품>위생용품>면도위생용품>면도기,면도기
5,9312529000623,마카다미아 쿠키,자점,자점,자점,1151551055,401,일반,예,20140101,...,0,0,0,0,0,NaN,NaN,마카다미아,신선식품>농산물>견과류>마카다미아,마카다미아
6,9312529000616,마카다미아 쿠키,자점,자점,자점,1151551055,401,일반,예,20140101,...,0,0,0,0,0,NaN,NaN,마카다미아,신선식품>농산물>견과류>마카다미아,마카다미아
7,8888826020905,질레트)일회용 면도기,자점,자점,자점,1151551055,401,일반,예,20140101,...,0,0,0,0,0,NaN,NaN,면도기,일상용품>위생용품>면도위생용품>면도기,면도기
8,8888826016175,질레트)벡토플러스면도기,자점,자점,자점,1151551055,401,일반,예,20140101,...,0,0,0,0,0,NaN,NaN,면도기,일상용품>위생용품>면도위생용품>면도기,면도기
9,8888826007821,질레트)콘투어플러스면도기,자점,자점,자점,1151551055,401,일반,예,20140101,...,0,0,0,0,0,NaN,NaN,면도기,일상용품>위생용품>면도위생용품>면도기,면도기


## Convert dataframe to CSV file

In [17]:
#new_item_df.to_csv("/home/jun0/Projects/emart24/emart24_codes/LYH/item_category_matched.csv", encoding = 'utf-8-sig',index = False) # without 상품예시 column
new_item_df.to_csv("/home/jun0/Projects/emart24/emart24_codes/LYH/item_category_matched_with_example.csv", encoding = 'utf-8-sig',index = False)  # with 상품예시 column

## Statistics of created DataFrame

In [18]:
import numpy as np
import pandas as pd
from pathlib import Path
import json
import re
import sys
from itertools import chain
from tqdm import tqdm
import re

# load new_item_df from csv
# 위의 코드들 사용하지 않고 위에서 생성된 결과물 new_item_df를 load하기 위한 코드
new_item_df = pd.read_csv("/home/jun0/Projects/emart24/emart24_codes/LYH/item_category_matched_with_example.csv")
new_item_df['점포발주입수수량'] = new_item_df['점포발주입수수량'].fillna(0)
new_item_df['공급가'] = new_item_df['공급가'].fillna(0).apply(lambda x: round(float(x)))
new_item_df['매가'] = new_item_df['매가'].fillna(0)
new_item_df['현재분류명'] = new_item_df['현재분류명'].fillna("")
#new_item_df['전체분류'] = new_item_df['전체분류'].fillna("")
#new_item_df['상품예시'] = new_item_df['상품예시'].fillna("")
new_item_df['점포유통기한일수'] = new_item_df['점포유통기한일수'].fillna(0)
new_item_df = new_item_df.astype({'점포발주입수수량': int, '공급가': int, '매가': int, '점포유통기한일수': int})
new_item_df.head(20)

/home/yoonho/anaconda3/envs/models/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PLU_CD,상품명,대분류명,중분류명,소분류명,카테고리코드,상품속성코드,상품속성명,판매가능여부,시작일자,...,WED_YN,THU_YN,FRI_YN,SAT_YN,SUN_YN,GDS_DSCRT,GDS_SLPNT_DSCRT,현재분류명,전체분류,상품예시
0,9414453003319,KGB 크랜베리맛,자점,자점,자점,1151551055,401,일반,예,20140101,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,,NaN,NaN
1,9331275008295,레돈도)카푸치노125g,과자/빵,과자행사,비스켓행사(빼빼로),2092112004,401,일반,예,20200918,...,1.0,1.0,1.0,1.0,0.0,1. 20년 빼빼로데이 기획상품 2. 틴 케이스에 웨이퍼롤이 담겨 있는 상품,1. 틴 케이스에 웨이퍼롤이 담겨 있는 상품,,NaN,NaN
2,9331275008271,수입)레돈도(바닐라)150g,자점,자점,자점,1151551055,401,일반,예,20140101,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,,NaN,NaN
3,9331275002996,레돈도딸기&크림,자점,자점,자점,1151551055,401,일반,예,20140101,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,딸기,신선식품>농산물>과일류>딸기,딸기
4,9312931607557,쉬크)퀴트로4중 면도날(4입),자점,자점,자점,1151551055,401,일반,예,20140101,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,면도날,일상용품>위생용품>면도위생용품>면도날,면도날


In [13]:
# load original data item dataframe

ROOT = Path('/home/jun0/Projects/emart24')
item = pd.read_csv(ROOT / 'data/prep/02.설명이추가된 상품마스터_20210325.csv', sep='|', dtype=str)
item['점포발주입수수량'] = item['점포발주입수수량'].fillna(0)
item['공급가'] = item['공급가'].fillna(0).apply(lambda x: round(float(x)))
item['매가'] = item['매가'].fillna(0)
item['점포유통기한일수'] = item['점포유통기한일수'].fillna(0)
item = item.astype({'점포발주입수수량': int, '공급가': int, '매가': int, '점포유통기한일수': int})
item.head()

,PLU_CD,상품명,대분류명,중분류명,소분류명,카테고리코드,상품속성코드,상품속성명,판매가능여부,시작일자,...,LT,MON_YN,TUE_YN,WED_YN,THU_YN,FRI_YN,SAT_YN,SUN_YN,GDS_DSCRT,GDS_SLPNT_DSCRT
0,9414453003319,KGB 크랜베리맛,자점,자점,자점,1151551055,401,일반,예,20140101,...,1,0,0,0,0,0,0,0,NaN,NaN
1,9331275008295,레돈도)카푸치노125g,과자/빵,과자행사,비스켓행사(빼빼로),2092112004,401,일반,예,20200918,...,1,1,1,1,1,1,1,0,1. 20년 빼빼로데이 기획상품 2. 틴 케이스에 웨이퍼롤이 담겨 있는 상품,1. 틴 케이스에 웨이퍼롤이 담겨 있는 상품
2,9331275008271,수입)레돈도(바닐라)150g,자점,자점,자점,1151551055,401,일반,예,20140101,...,1,0,0,0,0,0,0,0,NaN,NaN
3,9331275002996,레돈도딸기&크림,자점,자점,자점,1151551055,401,일반,예,20140101,...,1,0,0,0,0,0,0,0,NaN,NaN
4,9312931607557,쉬크)퀴트로4중 면도날(4입),자점,자점,자점,1151551055,401,일반,예,20140101,...,1,0,0,0,0,0,0,0,NaN,NaN


In [14]:
print("original data cases: ",len(item))
print("processed data cases: ", len(new_item_df))
print("number of cases that does not have corresponding 상품예시: ",sum(new_item_df["상품예시"] == ""))

original data cases:  49299
processed data cases:  57646
number of cases that does not have corresponding 상품예시:  28381
